In [2]:
import os, datetime
# os.system("activate usc39")
DATETIME_NOW = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
SEED = 1

DATASET_DIR = "C:/Users/User/Documents/cer_dataset_16k_resampled_split/"
EPOCHS = 20
MODEL_NAME = "YAMNET"
SEED = 1
FILE_RATIO = 0.01
PATCH_HOP_DISTANCE = 0.25
    
import os, sys, glob
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import random
# import datetime

sys.path.append('src')
from models.get_models import get_model, model_builder_yamnet, model_builder_vggish
import models.yamnet_tf2.params as params
params = params.Params(sample_rate=16000, patch_hop_seconds=PATCH_HOP_DISTANCE) # 0.25

# from dataload_utils.data_load import get_dataset, get_filenames_and_classnames_list
import dataload_utils.data_load as data_load
from dataload_utils.data_aug import mix_up

# SEED = 42
random.seed(SEED)
tf.random.set_seed(SEED)

# parent_dir = "C:\\Users\\User\\Documents\\cer_dataset_16k_flattened_resampled\\"
dataset_loader = data_load.Dataset_loader(DATASET_DIR, params)
filenames_all = dataset_loader.__filenames_all__
classes = dataset_loader.__classes__
num_classes = dataset_loader.__num_classes__
print("classes: {}, num_classes: {}".format(classes, num_classes))

# To do real shuffling
AUTOTUNE = tf.data.AUTOTUNE
batch_size=64
random.shuffle(filenames_all)
filenames_all=filenames_all[:int(len(filenames_all)*FILE_RATIO)]
filenames_train = filenames_all[:int(len(filenames_all)*0.7)]
filenames_eval = filenames_all[int(len(filenames_all)*0.7):int(len(filenames_all)*0.9)]
filenames_test = filenames_all[int(len(filenames_all)*0.9):]

# Training set preparation
dataset_aug = dataset_loader.get_dataset(filenames_train, augment=True)
train_dataset = dataset_aug.shuffle(batch_size*2).batch(batch_size) # Batch before doing mixup

# Mixup -
random.shuffle(filenames_train)
dataset_no_aug = dataset_loader.get_dataset(filenames_train, augment=False)

zipped_ds = tf.data.Dataset.zip((
    dataset_aug.shuffle(batch_size*2).batch(batch_size), 
    dataset_no_aug.shuffle(batch_size*2).batch(batch_size)
    ))

train_dataset = zipped_ds.map(
    map_func = lambda ds_one, ds_two: mix_up(ds_one, ds_two, alpha=0.2), 
    num_parallel_calls=AUTOTUNE
    )

eval_dataset = dataset_loader.get_dataset(filenames_eval, augment=False).shuffle(batch_size*2).batch(batch_size)
test_dataset = dataset_loader.get_dataset(filenames_test, augment=False, flat_map=False).shuffle(batch_size*2)#.batch(batch_size)

train_dataset = train_dataset.cache().prefetch(AUTOTUNE)
eval_dataset = eval_dataset.cache().prefetch(AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(AUTOTUNE)

# length = len(list(dataset_train_eval))
# print("Total length of dataset: ", length)

# Paths
training_path = "./training/{}".format(DATETIME_NOW)

model_training_path = training_path + "/{}".format(MODEL_NAME)
ckp_path = model_training_path + "/checkpoints/cp.ckpt"
log_path = model_training_path + "/logs/fit"    
hd5_path = model_training_path + "/model.hd5"
cfm_path = model_training_path + "/confusion_matrix.png"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                save_weights_only=True,
                                                verbose=1)
# Create a tensorboard callback                         
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_path, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

import keras_tuner as kt
tuner = kt.Hyperband(model_builder_yamnet,
                objective='val_accuracy',
                max_epochs=10,
                factor=3,
                directory='my_dir',
                project_name='intro_to_kt')    

tuner.search(train_dataset, validation_data = eval_dataset, epochs=50, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. 
Optimal Dense: {best_hps.get('units')} 
Optimal Dropout: {best_hps.get('dropout')}
Optimal lr for optimizer: {best_hps.get('learning_rate')}.
""")

model = tuner.hypermodel.build(best_hps)
# Fit model from scratch

if MODEL_NAME=="YAMNET" or MODEL_NAME=="VGGISH":
    # Transfer learn
    # make all layers untrainable by freezing weights (except for last layer)
    for l, layer in enumerate(model.layers[:-7]):
        layer.trainable = False

    # First first time
    model.fit(train_dataset, validation_data = eval_dataset, epochs=EPOCHS-5, 
        verbose=1, callbacks=[cp_callback,tensorboard_callback])
    
    # unfreeze all layers
    for l, layer in enumerate(model.layers[:-7]):
        layer.trainable = True
        
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
        loss=tf.keras.losses.CategoricalCrossentropy(), #from_logits=True
        metrics=['accuracy'],
    )
    model.fit(train_dataset, validation_data = eval_dataset, initial_epoch = EPOCHS-5, epochs=EPOCHS, 
        verbose=1, callbacks=[cp_callback,tensorboard_callback])
    
else:
    # Fit model from scratch
    model.fit(train_dataset, validation_data = eval_dataset, epochs=EPOCHS, 
        verbose=1, callbacks=[cp_callback,tensorboard_callback])

# Evaluate performance of model with test fold (that it wasn't trained on)
model.load_weights(ckp_path)
loss, acc = model.evaluate(test_dataset, verbose=2)



Number of files in Class_00_Modified_car_engines: 4644
Number of files in Class_01_Regular_Vehicles: 9539
Number of files in Class_02_Tools_and_Mechanisms: 13307
Number of files in Class_03_Environmental_Sounds: 10488
Number of files:  37978
classes: ['Class_00_Modified_car_engines', 'Class_01_Regular_Vehicles', 'Class_02_Tools_and_Mechanisms', 'Class_03_Environmental_Sounds'], num_classes: 4
INFO:tensorflow:Reloading Oracle from existing project my_dir\intro_to_kt\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\intro_to_kt\tuner0.json
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. 
Optimal Dense: 640 
Optimal Dropout: 0.30000000000000004
Optimal lr for optimizer: 0.0001.

Epoch 1/15


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
# Get y_preds = predictions made by model
y_preds,y_trues = [],[]
for x_test, y_true in list(test_dataset):
    y_pred = np.argmax(model.predict(x_test), axis=1)
    y_true = np.argmax(y_true, axis=1)
    y_preds.extend(y_pred)
    y_trues.extend(y_true)
y_trues = np.array(y_trues)

y_preds = np.array(y_preds)

accuracy = accuracy_score(y_trues, y_preds)
print("Testing accuracy: ", accuracy)


cm, ax = plt.subplots(figsize=(10,10))
try:
    cm = ConfusionMatrixDisplay.from_predictions(
        y_trues, y_preds, normalize='true', 
        display_labels=classes, xticks_rotation=90,
        ax=ax
    )
except:
    cm = ConfusionMatrixDisplay.from_predictions(
        y_trues, y_preds, normalize='true', 
        xticks_rotation=90,
        ax=ax
    )
ax.set_title("{}, Acc: {:02f}".format(model_training_path.split("/")[-1], accuracy))
cm.figure_.savefig(cfm_path,dpi=300)